# Demo

###### Imports

In [2]:
# This Python file uses the following encoding: utf-8
from lib.predict_pdf import predict_documents_pdf, predictions_dataframe, load_prediction_models
from lib.utils import move_flattened_files
from lib.untar import ExtractNested
import os

###### Input arguments

In [3]:
pdf_folder = '../files_to_predict'
csv_out_file = '../predictions2.csv'

do_ocr = False
extract_from_tar = False
tar_root = '../prova.tar.gz'
use_a05_model = True

###### Tar extraction (execute only the first time if starting from a tar)

In [ ]:
# Accetto solo file con dimensione < 1MB e che finiscono per 001.pdf (si può cambiare a piacere il filtro qui)
def file_filter(f):
    return str(f).endswith(u'001.pdf') and os.path.getsize(str(f))<(1024**2)

In [ ]:
# Solo se si deve partire da una tar e non si è già estratto
if extract_from_tar and not os.path.exists(tar_root[:-7]):
    ExtractNested(tar_root)
    move_flattened_files(tar_root[:-7], pdf_folder, file_filter)

###### Load models

In [4]:
models_original = {'gensim_file':'models/gensim_model_5000.d2v', 
                  'keras_model_file':'models/keras_model.json',
                  'keras_weights_file':'models/keras_weights_5000.h5',
                  'permitted_words_file':'first_5000_words.json'}

models_with_a05 = {'gensim_file':'models/gensim_5000_model_with_verb.d2v', 
                  'keras_model_file':'models/keras_model.json',
                  'keras_weights_file':'models/keras_new_weights_with_verb_es.h5',
                  'permitted_words_file':'first_5000_words_with_verb_cost.json'}

models_with_a05_2 = {'gensim_file':'models/gensim_5000_model_with_verb.d2v', 
                  'keras_model_file':'models/keras_model_verb.json',
                  'keras_weights_file':'models/keras_verb_checkpoint.h5',
                  'permitted_words_file':'first_5000_words_with_verb.json'}

model_with_a05_experimental = {'gensim_file':'models/gensim_5000_model_with_verb.d2v', 
                  'keras_model_file':'models/keras_model_retry3.json',
                  'keras_weights_file':'models/keras_weights_verb_retry3.h5',
                  'permitted_words_file':'first_5000_words_with_verb_cost.json'}

# Change the model here
models_demo = model_with_a05_experimental if use_a05_model else models_original

predict_pdfs = load_prediction_models(**models_demo)

### Prediction

In [5]:
pdf_names = [os.path.join(pdf_folder, f) for f in os.listdir(pdf_folder) if f.lower().endswith("pdf")]
#pdf_names = ['../files_to_predict/4908476390001.pdf']
predictions, filtered_filenames = predict_pdfs(pdf_names, do_ocr=do_ocr)

#### Save predictions to csv

In [6]:
def basenames(files):
    return [os.path.basename(f) for f in files]

In [7]:
df = predictions_dataframe(basenames(pdf_names), basenames(filtered_filenames), predictions, csv_out_file)
#df.to_csv(csv_out_file,  sep=',', index=False, encoding='utf-8')

In [8]:
df

,Nome file,Errore,Messaggio errore,Output rete,Predizione
0,3297438520001.pdf,No,,0.019055,NON COSTITUTIVO
1,4908019380001.pdf,No,,0.448735,NON COSTITUTIVO
2,3297448050002.pdf,No,,0.000466395,NON COSTITUTIVO
3,4907876150001.pdf,No,,0.407489,NON COSTITUTIVO
4,4907903520001.pdf,No,,0.952048,COSTITUTIVO
5,4907874650001.pdf,No,,0.952569,COSTITUTIVO
6,3296082450001.pdf,No,,0.0010353,NON COSTITUTIVO
7,4908013560001.pdf,No,,0.990338,COSTITUTIVO
8,4907882890001.pdf,No,,0.995121,COSTITUTIVO
9,4907924550001.pdf,No,,0.978893,COSTITUTIVO


In [9]:
len(df.loc[df["Predizione"]=="NON COSTITUTIVO"])

28